### Finetuning on custom Data




In [15]:
from datasets import load_dataset
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load dataset 
data = pd.read_csv("intent_data.csv")

# Convert labels to numeric: 'informative' -> 0, 'transactional' -> 1
data['label'] = data['label'].map({'informative': 0, 'transactional': 1})

data.to_csv("encoded_intent_data.csv", index=False)

dataset = load_dataset('csv', data_files="encoded_intent_data.csv", split='train')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def tokenize_function(examples):
    return tokenizer(examples['query'], padding='max_length', truncation=True, max_length=128)

dataset = dataset.map(tokenize_function, batched=True)

# dataset


Generating train split: 140 examples [00:00, 456.63 examples/s]
e:\ps_project_task\research\.venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 140/140 [00:00<00:00, 1046.02 examples/s]


In [8]:
# Split the dataset into train and test (80% training, 20% testing)
train_dataset, test_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()

# Set dataset format for PyTorch (input_ids, attention_mask, and labels)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [9]:
# Load pre-trained BERT model for sequence classification (2 labels)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',                
    evaluation_strategy="epoch",          
    save_strategy="epoch",                
    learning_rate=2e-5,                    
    per_device_train_batch_size=16,        
    per_device_eval_batch_size=32,         
    num_train_epochs=3,                    
    weight_decay=0.01,                    
    logging_dir='./logs',                  
    logging_steps=10,
    load_best_model_at_end=True,           
)

# Initialize Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,           
)

# Train the model
trainer.train()


trainer.save_model("fine_tuned_bert_model")

# Evaluate 
results = trainer.evaluate()

# print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 7/21 [06:32<13:04, 56.02s/it]



                               
                                              
 33%|███▎      | 7/21 [06:00<00:53,  3.83s/it]
Checkpoint destination directory ./results\checkpoint-7 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.6006103754043579, 'eval_runtime': 2.1956, 'eval_samples_per_second': 12.298, 'eval_steps_per_second': 0.455, 'epoch': 1.0}



 33%|███▎      | 7/21 [06:18<00:53,  3.83s/it] 

{'loss': 0.632, 'learning_rate': 1.0476190476190477e-05, 'epoch': 1.43}





                               
                                               
 33%|███▎      | 7/21 [06:36<00:53,  3.83s/it]
Checkpoint destination directory ./results\checkpoint-14 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

{'eval_loss': 0.5232444405555725, 'eval_runtime': 2.051, 'eval_samples_per_second': 13.165, 'eval_steps_per_second': 0.488, 'epoch': 3.0}



100%|██████████| 21/21 [01:45<00:00,  5.04s/it]


{'train_runtime': 105.9181, 'train_samples_per_second': 3.031, 'train_steps_per_second': 0.198, 'train_loss': 0.595306260245187, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<00:00, 334.15it/s]

{'eval_loss': 0.5232444405555725, 'eval_runtime': 2.066, 'eval_samples_per_second': 13.069, 'eval_steps_per_second': 0.484, 'epoch': 3.0}


### Prediction

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained("fine_tuned_bert_model")  #aved model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Use the same tokenizer you used for training

def predict_intent(input_text):
    # Tokenize the input
    inputs = tokenizer(input_text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    
    # Set the model to evaluation mode (important for dropout layers during inference)
    model.eval()

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  

    # Get the predicted class (highest logit value)
    predicted_class = torch.argmax(logits, dim=-1)

    # Print the prediction
    if predicted_class == 0:
        print(f"Query: '{input_text}' is classified as 'informative'.")
    else:
        print(f"Query: '{input_text}' is classified as 'transactional'.")




e:\ps_project_task\research\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\ps_project_task\research\.venv\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
e:\ps_project_task\research\.venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
e:\ps_project_task\research\.venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` in

In [9]:

predict_intent("I want to add benificiary")

Query: 'I want to add benificiary' is classified as 'informative'.


In [89]:
# import torch
# import torch.nn.functional as F

# def get_feedback(query, model, tokenizer):
#     # Preprocess the query and get model prediction
#     inputs = tokenizer(query, padding=True, truncation=True, max_length=128, return_tensors="pt")
#     model.eval()
    
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
    
#     # Apply softmax to get probabilities
#     probs = F.softmax(logits, dim=-1)
#     predicted_class = torch.argmax(logits, dim=-1)
    
#     print(f"Model Prediction: {'informative' if predicted_class == 0 else 'transactional'} (Confidence: {probs[0][predicted_class].item():.4f})")
    
#     # If confidence is below a threshold, ask for user feedback
#     if probs[0][predicted_class].item() < 0.7:  # Example threshold for low confidence
#         user_feedback = input("Is this correct? (yes/no): ")
#         if user_feedback.lower() != 'yes':
#             correct_label = input("Please enter the correct label (informative / transactional): ")
#             return query, correct_label  # Store this for retraining
#     return None


In [90]:
# import pandas as pd


# def save_feedback_to_csv(query, correct_label, feedback_file="intent_data.csv"):
 
#     try:
#         feedback_data = pd.read_csv(feedback_file)
#     except FileNotFoundError:
#         feedback_data = pd.DataFrame(columns=["query", "label"])
    
#     # Add new feedback to the dataset
#     new_feedback = pd.DataFrame({"query": [query], "label": [correct_label]})
#     feedback_data = pd.concat([feedback_data, new_feedback], ignore_index=True)
    
#     # Save the updated feedback data back to CSV
#     feedback_data.to_csv(feedback_file, index=False)


### Testing

In [3]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F


model = BertForSequenceClassification.from_pretrained("fine_tuned_bert_model")  
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  

def preprocess_input(input_text,tokenizer):
    # Tokenize the input
    inputs = tokenizer(input_text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    return inputs


def save_feedback_to_csv(query, correct_label, feedback_file):
 
    try:
        feedback_data = pd.read_csv(feedback_file)
    except FileNotFoundError:
        feedback_data = pd.DataFrame(columns=["query", "label"])
    
    
    new_feedback = pd.DataFrame({"query": [query], "label": [correct_label]})
    feedback_data = pd.concat([feedback_data, new_feedback], ignore_index=True)
    
    feedback_data.to_csv(feedback_file, index=False)

def get_feedback(query, model , tokenizer, feedback_file, confidence_threshold=0.8): # feedback_file="intent_data.csv"
    # Preprocess the query and get model prediction
    inputs = preprocess_input(query,tokenizer=tokenizer)
    model.eval()
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1)
    predicted_class = torch.argmax(logits, dim=-1)
    
    print(f"Model Prediction: {'informative' if predicted_class == 0 else 'transactional'} (Confidence: {probs[0][predicted_class].item():.4f})")
    
    prediction_list = []
        # prediction
    if predicted_class == 0:
        prediction_list.append('informative')
        prediction_list.append('transactional')
    else:
        prediction_list.append('transactional')
        prediction_list.append('informative')
  
    if probs[0][predicted_class].item() < confidence_threshold: 
        user_feedback = input("Is this prediction correct? ( Y/N ): ")
        if user_feedback.lower() == 'y':
            save_feedback_to_csv(query=query, correct_label= prediction_list[0] , feedback_file= feedback_file)
            print(f"Thank you for the feedback! As per feedback {prediction_list[0].upper()} intent has been saved.")
        
        elif user_feedback.lower() == 'n':
            save_feedback_to_csv(query, prediction_list[1] , feedback_file)
            print(f"Thank you for the feedback! As per feedback {prediction_list[1].upper()} intent has been saved.")

        else:
            print("Not valid input.")


e:\ps_project_task\research\.venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
## TESTING
get_feedback(query = input("Enter Query : "), 
             model=model, tokenizer=tokenizer, 
             feedback_file="intent_data.csv", 
             confidence_threshold=0.8)

Model Prediction: transactional (Confidence: 0.5062)
Thank you for the feedback! TRANSACTIONAL Intent has been saved.


###  Finetuining on feedback data 

In [ ]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer

def retrain_model_with_feedback(feedback_file="feedback_data.csv"):
    # Load feedback data
    feedback_data = pd.read_csv(feedback_file)
    
    # Tokenize the new feedback data
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    def tokenize_function(examples):
        return tokenizer(examples['query'], padding=True, truncation=True, max_length=128)
    
    feedback_dataset = Dataset.from_pandas(feedback_data)
    feedback_dataset = feedback_dataset.map(tokenize_function, batched=True)
    
    # Load your pre-trained model
    model = BertForSequenceClassification.from_pretrained("fine_tuned_bert_model", num_labels=2)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
    )
    
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=feedback_dataset,  # Use feedback data for training
    )
    
    # Retrain the model with the new data
    trainer.train()

    # Save the updated model
    trainer.save_model("fine_tuned_bert_model")
